<a href="https://colab.research.google.com/github/HoangShiya/HUET/blob/main/Th%E1%BB%B1c_h%C3%A0nh_ki%E1%BB%83m_%C4%91%E1%BB%8Bnh_c%C3%A1c_t%C3%ADnh_ch%E1%BA%A5t_c%E1%BB%A7a_d%E1%BB%AF_li%E1%BB%87u_d%C3%B9ng_trong_ph%C3%A2n_t%C3%ADch_ANOVA_1_chi%E1%BB%81u.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thực hành kiểm định các tính chất của dữ liệu dùng trong phân tích ANOVA 1 chiều


---

## PHÂN TÍCH ANOVA MỘT CHIỀU

Phân tích ANOVA chấp nhận các giả định sau:

> (1) Tất cả các nhóm đều độc lập với nhau

> (2) Dữ liệu của mỗi nhóm đều tuân theo phân phối chuẩn. Tính chất này có thể được kiểm định bằng phương pháp Shapiro-Wilks.

> (3) Phương sai (variance) của các nhóm bằng nhau. Tính chất này có thể được kiểm định bằng phương pháp kiểm định Bartlett hay Levene.

Nếu dữ liệu hiện có thỏa mãn các tính chất trên thì chúng ta mới tiến hành kiểm định ANOVA. Do vậy trước khi tiến hành kiểm định ANOVA, chúng ta cần thực hiện kiểm tra tập dữ liệu cần thỏa mãn 3 tính chất trên.

---

    

Import các thư viện cần thiết để tiến hành phân tích Anova

In [2]:
!pip install -U xlrd pyxlsb
import numpy as np
import pandas as pd
import scipy.stats as st

> **Bước 1**: Khởi tạo một tập dữ liệu mẫu sample_1 với kích thước 100 tuân theo phân phối chuẩn

In [20]:
# Khởi tạo giá trị trung bình, sigma, kích thước của tập dữ liệu
mean = 3.
sig = 1.
size = 100

# Khởi tạo một trình tạo mới BitGenerator mặc định (PCG64)
rng = np.random.default_rng()

# Khởi tạo mẫu sample_1 theo phân phối chuẩn: giá trị trung bình 3., sigma 1., kích thước 100 phần tử
sample_1 = st.norm.rvs(mean, sig, size, random_state = rng)
print(f"Giá trị 100 phần tử tuân theo phân phối chuẩn của mẫu sample 1:\n {sample_1}")

Giá trị 100 phần tử tuân theo phân phối chuẩn của mẫu sample 1:
 [2.36268883 4.19554736 1.15859172 3.44985223 2.50706856 4.92194592
 2.74388592 0.80235123 1.63087787 4.02064881 2.6117931  3.18264731
 2.14579537 3.31091914 3.76733313 2.12741871 3.14648108 1.7792325
 4.21987499 2.9965771  3.08056989 2.2051788  2.73038573 4.47395849
 3.4725546  1.78410305 4.61959964 2.74345141 5.29775818 3.64581009
 1.65058099 4.00656062 1.36108772 3.0444376  3.04384913 2.6893849
 2.90680361 3.20306591 3.2455277  3.26263801 4.73623168 3.79411862
 2.37911789 1.97097057 2.95957086 4.12752544 2.39329146 3.03243428
 2.83738674 4.85220455 2.60455057 2.00380905 3.89997399 4.02904036
 4.5728625  1.97132902 1.63603599 2.6598201  5.08703783 2.9131223
 1.13356321 3.71793301 3.76903626 3.84761411 2.92152335 3.66703685
 3.60836412 3.7227892  0.03722038 2.50798276 3.1071164  2.78111301
 2.92909949 3.12222666 3.8417195  1.98063145 1.9902874  2.61335202
 1.82084632 3.12396079 3.31634704 4.90039935 2.61847226 2.60748756


> **Bước 2**: hãy tạo tập dữ liệu mẫu sample_2 với kích thước 100 tuân theo phân phối Poisson

In [17]:
# Khởi tạo giá trị lam, kích thước của tập dữ liệu
# lam: Số lượng sự kiện dự kiến ​​xảy ra trong một khoảng thời gian cố định, phải >= 0. Một chuỗi phải được phát trên kích thước được yêu cầu.
lam = 3
size = 100

# Khởi tạo mẫu sample_2 theo phân phối poisson: giá trị lam 3, kích thước 100 phần tử
sample_2 = np.random.poisson(lam, size)
print(f"Giá trị 100 phần tử tuân theo phân phối poisson của mẫu sample 2:\n {sample_2}")



Giá trị 100 phần tử tuân theo phân phối poisson của mẫu sample 2:
 [5 4 3 2 4 2 1 3 3 3 3 2 1 2 1 5 2 2 1 1 6 5 3 3 0 4 4 2 0 2 1 1 2 2 3 7 1
 3 3 3 4 1 6 6 2 3 4 3 0 3 7 3 0 3 3 2 4 5 2 1 2 0 4 1 1 5 5 3 3 3 2 5 3 3
 4 2 6 1 1 2 6 6 3 2 4 1 2 4 3 2 4 2 2 1 3 4 1 2 1 3]


## Kiểm định tính chất thứ 2 bằng phương pháp Shapiro-Wilks đối với sample_1 và sample_2

> **Bước 1**: Khởi tạo dữ liệu và tính toán các thông số cần thiết

In [36]:
# Dữ liệu được sử dụng là sample_1 và sample_2
print("Các dữ liệu được sử dụng để kiểm định tính chất thử 3:\n sample_1 theo phân phối chuẩn.\n sample_2 theo phân phối poisson")

# Khởi tạo biến alpha lưu trữ mức ý nghĩa cho toàn bộ quá trình phân tích
alpha = 0.05
print(f"Mức ý nghĩa cho toàn bộ quá trình phân tích: {alpha}")

Các dữ liệu được sử dụng để kiểm định tính chất thử 3:
 sample_1 theo phân phối chuẩn.
 sample_2 theo phân phối poisson
Mức ý nghĩa cho toàn bộ quá trình phân tích: 0.05


> **Bước 2**: Xây dựng giả thuyết

Giả thuyết $H_0$:

$H_0$: Dữ liệu sau khi thực hiện kiểm tra Shapiro-Wilk bé hơn mức ý nghĩa cho toàn bộ quá trình phân tích

Giả thuyết $H_a$:

$H_a$: Dữ liệu sau khi thực hiện kiểm tra Shapiro-Wilk không bé hơn mức ý nghĩa cho toàn bộ quá trình phân tích

> **Bước 3**: Thực hiện tiến hành kiểm định và tính các giá trị liên quan

In [31]:
# Khởi tạo giá trị p và mẫu theo 
sp1, p1 = st.shapiro(sample_1)
sp2, p2 = st.shapiro(sample_2) 

print(f"Giá trị p của sample_1 sau khi kiểm định bằng phương pháp Shapiro-Wilk: {np.round(p1,4)}")
print(f"Giá trị p của sample_2 sau khi kiểm định bằng phương pháp Shapiro-Wilk: {np.round(p2,5)}")

Giá trị p của sample_1 sau khi kiểm định bằng phương pháp Shapiro-Wilk: 0.9336
Giá trị p của sample_2 sau khi kiểm định bằng phương pháp Shapiro-Wilk: 0.0002


> **Bước 4**: Kết luật kết quả sau khi kiểm định

In [34]:
def test_conclusion(p, alpha):
  if (p < alpha):
    print(f"Vì giá trị p thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định ({np.round(p, 5)} < {alpha}) nên không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p không thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định ({np.round(p, 5)} >= {alpha}) nên có bằng chứng bác bỏ giả thuyết H0")


# Kết luận kết quả kiểm định dữ liệu sample_1
print("DỮ LIỆU SAMPLE_1")
test_conclusion(p1, alpha)
print("\n")

# Kết luận kết quả kiểm định dữ liệu sample_2
print("DỮ LIỆU SAMPLE_2")
test_conclusion(p2, alpha)




DỮ LIỆU SAMPLE_1
Vì giá trị p không thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định (0.93355 >= 0.05) nên có bằng chứng bác bỏ giả thuyết H0


DỮ LIỆU SAMPLE_2
Vì giá trị p thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định (0.0002 < 0.05) nên không có bằng chứng bác bỏ giả thuyết H0


## Kiểm định tính chất thứ 3 bằng phương pháp Bartlett và Levene đối với sample_1 và sample_2

> **Bước 1**: Khởi tạo dữ liệu và tính toán các thông số cần thiết

In [37]:
# Dữ liệu được sử dụng là sample_1 và sample_2
print("Các dữ liệu được sử dụng để kiểm định tính chất thử 3:\n sample_1 theo phân phối chuẩn.\n sample_2 theo phân phối poisson")

# Khởi tạo biến alpha lưu trữ mức ý nghĩa cho toàn bộ quá trình phân tích
alpha = 0.05
print(f"Mức ý nghĩa cho toàn bộ quá trình phân tích: {alpha}")

Các dữ liệu được sử dụng để kiểm định tính chất thử 3:
 sample_1 theo phân phối chuẩn.
 sample_2 theo phân phối poisson
Mức ý nghĩa cho toàn bộ quá trình phân tích: 0.05


> **Bước 2**: Xây dựng giả thuyết

Giả thuyết $H_0$:

$H_0$: Phương sai giữa các lựa chọn của biến định tính ở trên khác nhau

Giả thuyết $H_a$:

$H_a$: Phương sai giữa các lựa chọn của biến định tính ở trên không khác nhau

> **Bước 3**: Thực hiện tiến hành kiểm định và tính các giá trị liên quan

In [41]:
# Tính giá trị p và stat bằng kiểm định Barlett của sample_1 và sample_2
p, stat = st.bartlett(sample_1, sample_2)

# Xuất kết quả ra màn hình:
print("GIÁ TRỊ CÁC BIẾN STAT, P SAU KHI KIỂM ĐỊNH BARTLETT")
print("---------------------------------------------------")
print(f"Giá trị stat: {np.round(stat, 10)}")
print(f"Giá trị p: {np.round(p, 2)}")

GIÁ TRỊ CÁC BIẾN STAT, P SAU KHI KIỂM ĐỊNH BARtLETT
---------------------------------------------------
Giá trị stat: 9.3553e-06
Giá trị p: 19.64


In [43]:
# Tính giá trị p và stat bằng kiểm định Levene của sample_1 và sample_2
p, stat = st.levene(sample_1, sample_2)

# Xuất kết quả ra màn hình:
print("GIÁ TRỊ CÁC BIẾN STAT, P SAU KHI KIỂM ĐỊNH BARTLETT")
print("---------------------------------------------------")
print(f"Giá trị stat: {np.round(stat, 5)}")
print(f"Giá trị p: {np.round(p, 2)}")

GIÁ TRỊ CÁC BIẾN STAT, P SAU KHI KIỂM ĐỊNH BARTLETT
---------------------------------------------------
Giá trị stat: 0.00025
Giá trị p: 13.94


> **Bước 4**: Kết luật kết quả sau khi kiểm định

In [45]:
if (p < alpha):
  print(f"Vì giá trị p thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định ({np.round(p, 2)} < {alpha}) nên không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p không thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định ({np.round(p, 2)} >= {alpha}) nên có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p không thấp hơn mức ý nghĩa của toàn bộ quá trình kiểm định (13.94 >= 0.05) nên có bằng chứng bác bỏ giả thuyết H0
